## GoogLeNet model for training Food101 datasets
* number of parameters: 10,621,391

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

from torchinfo import summary

from pathlib import Path

import sys
sys.path.append('..')

from python_scripts import model_builder, engine, data_setup

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
food101_transforms_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

food101_transforms_test = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

In [4]:
learning_rate_list = [1e-3]
weight_decay_list = [1e-3]
epochs_list = [5]
batch_size_list = [16]

In [5]:
data_dir = Path('../data/extracted')

train_data_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=food101_transforms_train,
    download=True
)
test_data_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=food101_transforms_test,
    download=True
)

class_names_food101 = train_data_food101.classes

In [6]:
def googlenet_model_generator(weights=None):
    return model_builder.GoogLeNet(num_features=len(class_names_food101))

In [7]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=0.1, # you can start with small number of train set to check normality at first
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=0.1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=None,
    model_generator=googlenet_model_generator,
    model_weights=None,
    model_name='GoogLeNet_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=False,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Splitting dataset of length 75750 into splits of size: 7575 and 68175
[INFO] Splitting dataset of length 25250 into splits of size: 2525 and 22725


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 4.7064, Train_acc: 0.0117 | Test_loss: 4.6122, Test_acc: 0.0202
Epoch: 1 | Train_loss: 4.6255, Train_acc: 0.0137 | Test_loss: 4.6465, Test_acc: 0.0131
Epoch: 2 | Train_loss: 4.6210, Train_acc: 0.0109 | Test_loss: 4.6236, Test_acc: 0.0079
Epoch: 3 | Train_loss: 4.6168, Train_acc: 0.0124 | Test_loss: 4.6162, Test_acc: 0.0091
Epoch: 4 | Train_loss: 4.6057, Train_acc: 0.0116 | Test_loss: 4.5975, Test_acc: 0.0119
